In [1]:
import stroll.util
from stroll.conllu import ConlluDataset
import torch
import stroll.labels
import stroll.graph
import stroll.model
import stroll.srl
import dgl

/home/dafne/anaconda3/envs/e2etest/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
Using backend: pytorch


In [2]:
dataset = stroll.conllu.ConlluDataset('example.conllu')

In [3]:
fname_fasttext, fname_model = stroll.util.download_srl_model('data')

In [4]:
state_dict = torch.load(fname_model)
hyperparams = state_dict.pop('hyperparams')

In [5]:
sentence_encoder = stroll.labels.FasttextEncoder(str(fname_fasttext))
dataset_graph = stroll.graph.GraphDataset(
            dataset=dataset,
            sentence_encoder=sentence_encoder,
            features=hyperparams.features
            )

In [6]:
dataloader = torch.utils.data.DataLoader(
            dataset_graph,
            #batch_size=50,
            #num_workers=2
            collate_fn=dgl.batch
            )

In [7]:
net = stroll.model.Net(
            in_feats=dataset_graph.in_feats,
            h_layers=hyperparams.h_layers,
            h_dims=hyperparams.h_dims,
            out_feats_a=2,
            out_feats_b=19,
            activation='relu'
            )

In [8]:
net.load_state_dict(state_dict)

stroll.srl.infer(net, dataloader, dataset_graph, batch_size=50)

/home/dafne/anaconda3/envs/e2etest/lib/python3.8/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)
/home/dafne/anaconda3/envs/e2etest/lib/python3.8/site-packages/dgl/base.py:45: DGLWarning: DGLGraph.__len__ is deprecated.Please directly call DGLGraph.number_of_nodes.
  return warnings.warn(message, category=category, stacklevel=1)


In [9]:
# Show the CONLL output including the SRL output
dataset

# newdoc id = text-stanza.conllu
# sent_id = 0
# text = Dit is een stukje tekst.
1	Dit	dit	PRON	VNW|aanw|pron|stan|vol|3o|ev	Person=3|PronType=Dem	4	nsubj	_	start_char=0|end_char=3	_	_	_
2	is	zijn	AUX	WW|pv|tgw|ev	Number=Sing|Tense=Pres|VerbForm=Fin	4	cop	_	start_char=4|end_char=6	_	_	_
3	een	een	DET	LID|onbep|stan|agr	Definite=Ind	4	det	_	start_char=7|end_char=10	_	_	_
4	stukje	stuk	NOUN	N|soort|ev|dim|onz|stan	Gender=Neut|Number=Sing	0	root	_	start_char=11|end_char=17	_	_	_
5	tekst	tekst	NOUN	N|soort|ev|basis|zijd|stan	Gender=Com|Number=Sing	4	nmod	_	start_char=18|end_char=23	_	_	_
6	.	.	PUNCT	LET	_	4	punct	_	start_char=23|end_char=24	_	_	_

# sent_id = 1
# text = Het dient als voorbeeld.
1	Het	het	PRON	VNW|pers|pron|stan|red|3|ev|onz	Person=3|PronType=Prs	2	nsubj	_	start_char=25|end_char=28	_	_	_
2	dient	dienen	VERB	WW|pv|tgw|met-t	Number=Sing|Tense=Pres|VerbForm=Fin	0	root	_	start_char=29|end_char=34	_	_	_
3	als	als	ADP	VZ|init	_	4	mark	_	start_char=35|end_char=38	_	_	_
4	voorbeeld